In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel, Field
import operator


In [2]:
load_dotenv()

True

In [3]:
model = AzureChatOpenAI(model='gpt-4o-mini', temperature=0.1)

In [4]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description="Detailed feedback for the essay.")
    score: int = Field(description="Score out of 10 for the essay.", ge=0, le=10)

In [5]:
structured_model = model.with_structured_output(EvaluationSchema)

In [6]:
essay="""
Artificial Intelligence (AI) is changing the way students learn and teachers teach.

It helps personalize learning for every student based on their speed and style.

AI tools can check homework and grade tests faster and more accurately.

Virtual tutors and chatbots give students help anytime they need it.

Teachers use AI to track student progress and find where they need support.

AI-powered apps make learning fun through games and interactive lessons.

It removes language barriers with real-time translation and voice support.

Students can explore subjects deeply using AI-powered research tools.

AI helps schools run better by managing attendance and scheduling.

Overall, AI makes education smarter, faster, and more accessible for everyone.

"""

In [7]:
structured_model.invoke(f"Evaluate the language quality of the essay and provide a feedback and assign a score out of 10:\n\n{essay}")

EvaluationSchema(feedback='The essay presents a clear and concise overview of the impact of Artificial Intelligence on education. The language is straightforward and easy to understand, making it accessible to a wide audience. However, the essay lacks depth and elaboration on each point. While it lists several benefits of AI in education, it does not provide examples or evidence to support these claims, which would enhance the argument. Additionally, the essay could benefit from a more structured format, perhaps with an introduction and conclusion to frame the discussion. Overall, the language quality is good, but the content could be more developed.', score=6)

In [30]:
class UPSCState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [31]:
def evaluate_language(state: UPSCState) -> UPSCState:
    prompt = f"Evaluate the language quality of the essay and provide a feedback and assign a score out of 10:\n\n{state['essay']}"
    result = structured_model.invoke(prompt)
    return {'language_feedback':result.feedback, 'individual_scores':[result.score] }

def evaluate_analysis(state: UPSCState) -> UPSCState:
    prompt = f"Evaluate the depth of the analysis of the essay and provide a feedback and assign a score out of 10:\n\n{state['essay']}"
    result = structured_model.invoke(prompt)
    return {'analysis_feedback':result.feedback, 'individual_scores':[result.score] }

def evaluate_clarity(state: UPSCState) -> UPSCState:
    prompt = f"Evaluate the clarity of thought of the essay and provide a feedback and assign a score out of 10:\n\n{state['essay']}"
    result = structured_model.invoke(prompt)
    return {'clarity_feedback':result.feedback, 'individual_scores':[result.score] }

def evaluate_overall(state: UPSCState) -> UPSCState:
    prompt = f"Create a summarize feedback the essay \n language feedback: {state['language_feedback']} \n depth of analysis feedback: {state['analysis_feedback']} \n clarity feedback: {state['clarity_feedback']}"
    overall_feedback = model.invoke(prompt).content
    print(overall_feedback)
    overall_score = sum(state['individual_scores'])/len(state['individual_scores'])
    return {'overall_feedback': overall_feedback, 'avg_score': overall_score}

In [32]:
graph = StateGraph(UPSCState)

graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_clarity',evaluate_clarity)
graph.add_node('evaluate_overall',evaluate_overall)

graph.add_edge(START,'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_clarity')

graph.add_edge('evaluate_language','evaluate_overall')
graph.add_edge('evaluate_analysis','evaluate_overall')
graph.add_edge('evaluate_clarity','evaluate_overall')

graph.add_edge('evaluate_overall',END)

workflow = graph.compile()

In [33]:
initial_state: UPSCState = {'essay': essay}

output=workflow.invoke(initial_state)
print(output['overall_feedback'])

**Summary of Feedback on the Essay**

The essay effectively outlines the transformative impact of Artificial Intelligence (AI) on education, presenting its main ideas in a clear and accessible manner. Key benefits such as personalized learning, efficient grading, and enhanced student support are well-articulated. However, the analysis is somewhat superficial, lacking depth and complexity. It fails to address potential drawbacks of AI, such as data privacy concerns and the digital divide, and could benefit from specific examples or case studies to bolster its arguments.

While the structure is logical, it is described as simplistic, and incorporating transitions between ideas could improve the overall flow. Additionally, a concluding statement summarizing the overall impact of AI on education would enhance clarity and provide a stronger finish. Overall, the essay communicates its main ideas effectively but would be significantly improved by deeper analysis and more illustrative examples